# Optimización de Portafolios de Inversión

## Teoría Moderna del Portafolio (Markovitz)

### Portafolio de Mínima Varianza

In [47]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

# tickers de acciones chilenas
tickers = [
    "IAM.SN",         # Inversiones Aguas Metropolitanas S.A.
    "CONCHATORO.SN",  # Viña Concha y Toro S.A.
    "LTM.SN",         # LATAM Airlines Group S.A.
    "SONDA.SN",       # Sonda S.A.
    "BSANTANDER.SN",  # Banco Santander Chile
    "SALFACORP.SN",   # SalfaCorp S.A.
    "AGUAS-A.SN",     # Aguas Andinas S.A.
    "RIPLEY.SN",      # Ripley Corp S.A.
    "ENELAM.SN",      # Enel Américas S.A.
    "CMPC.SN",        # Empresas CMPC S.A.
    "BCI.SN",         # Banco de Crédito e Inversiones
    "CHILE.SN",       # Banco de Chile
    "COLBUN.SN",      # Colbún S.A.
    "ENELCHILE.SN",   # Enel Chile S.A.
    "ENTEL.SN",       # Empresa Nacional de Telecomunicaciones
    "FALABELLA.SN",    # Falabella S.A.     
    "SQM-B.SN"       # Sociedad Química y Minera de Chile S.A. 
]
df_prices = yf.download(tickers, start='2023-01-01', end='2025-12-31')['Close']

# Calcular retornos diarios
returns = df_prices.pct_change().dropna()

# Matriz de covarianza anualizada
cov_matrix = returns.cov() * 252

# Función objetivo: minimizar la varianza del portafolio
def portfolio_variance(weights, cov_matrix):
    contribution_vector = np.dot(weights, cov_matrix)
    variance = np.dot(contribution_vector, weights)
    return variance

# Restricciones: suma de pesos = 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Límites: pesos entre 0 y 1 (no hay ventas en corto)
bounds = tuple((0, 1) for _ in range(len(tickers)))

# Peso inicial igual para todos los activos
initial_weights = np.array(len(tickers) * [1. / len(tickers)])

# Optimización
result = minimize(portfolio_variance, initial_weights, args=(cov_matrix,),
                  method='SLSQP', bounds=bounds, constraints=constraints)   
min_variance_weights = result.x

df_min_variance = pd.DataFrame({'Ticker': tickers, 'Optimal Weight': min_variance_weights})   
df_min_variance.sort_values(by='Optimal Weight', ascending=False, inplace=True)
df_min_variance['Optimal Weight'] = df_min_variance['Optimal Weight'].apply(lambda x: f"{x:.2%}")
df_min_variance.reset_index(drop=True, inplace=True)
df_min_variance

[*********************100%***********************]  17 of 17 completed


,Ticker,Optimal Weight
0,SONDA.SN,13.89%
1,CHILE.SN,13.11%
2,AGUAS-A.SN,13.08%
3,FALABELLA.SN,12.60%
4,RIPLEY.SN,9.80%
5,IAM.SN,9.09%
6,SALFACORP.SN,7.62%
7,CMPC.SN,7.44%
8,ENTEL.SN,6.46%
9,COLBUN.SN,4.59%
